In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Project Title : Sentiment analysis on Israel Palestine Conflict using reddit
## API Script

In [2]:
!pip install praw
import praw
reddit = praw.Reddit(
    client_id = "get own api details from reddit",
    client_secret="get own api details from reddit",
    user_agent = "my_data")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 5.0 MB/s eta 0:00:00a 0:00:01


In [3]:
#the subreddit names to be changed based on sub reddit we use, i used the following subreddit for data sourcing
#“IsraelUnderAttack”, “IsraelHamas”, “IsraelPalestineWar_23”, hamasdeadlyattack”, “IsraelPalestine” ,“IsraelWar”.
#mention number of post required in total_limit
subreddit = reddit.subreddit("IsraelPalestine")
limit_per_request = 1000
total_limit = 10000
all_posts = []

# Fetch posts in chunks until the desired total limit is reached
while len(all_posts) < total_limit:
    # Calculate the remaining limit for the current request
    remaining_limit = total_limit - len(all_posts)
    
    # Adjust the limit for the current request based on the remaining limit
    current_limit = min(limit_per_request, remaining_limit)
    
    # Fetch the top posts using the current limit
    top_posts = subreddit.new(limit=current_limit)
    
    # Extend the list of all_posts with the current set of posts
    all_posts.extend(top_posts)
    
    # Print a message to indicate progress (optional)
    print(f"Fetched {len(all_posts)} out of {total_limit} posts")

Fetched 987 out of 10000 posts
Fetched 1974 out of 10000 posts
Fetched 2961 out of 10000 posts
Fetched 3948 out of 10000 posts
Fetched 4935 out of 10000 posts
Fetched 5922 out of 10000 posts
Fetched 6909 out of 10000 posts
Fetched 7896 out of 10000 posts
Fetched 8883 out of 10000 posts
Fetched 9870 out of 10000 posts
Fetched 10000 out of 10000 posts


In [4]:
len(all_posts)

10000

In [5]:
posts_data = []
count = 0
for post in all_posts:
    post_info = {
        
        "title": post.title,
        "score": post.score,
        "id": post.id,
        "url": post.url,
        "comms_num": post.num_comments,
        "created": post.created_utc,
        "body": post.selftext,
        "timestamp": post.created_utc,
        "subreddit" : post.subreddit.display_name
    }
    posts_data.append(post_info)
    count += 1
df = pd.DataFrame(posts_data)
print(df)


                                                  title  score       id  \
0                       IDF kills 3 hostages by mistake     27  18jabff   
1     the difficult and ongoing work of peace at the...     16  18j82h6   
2            I think Israel will be attacked again soon     11  18j7x6d   
3     As an Israeli leftist, my generation's discuss...    166  18j5trn   
4     Whats the point of issuing visa bans on violen...     18  18j51qb   
...                                                 ...    ...      ...   
9995  Hamas is misfiring rockets that land in Gaza, ...      0  18fghhl   
9996  Gaza artist going to Ireland ends up trapped i...      1  18fg57i   
9997                                   Genuine question     29  18feaw9   
9998                        Israel as the Jews of Islam      0  18fdo4b   
9999  Israel: 10-15% of Palestinian men stripped nak...      0  18fbqrz   

                                                    url  comms_num  \
0     https://www.reddit.com/

In [6]:
df.head()

,title,score,id,url,comms_num,created,body,timestamp,subreddit
0,IDF kills 3 hostages by mistake,27,18jabff,https://www.reddit.com/r/IsraelPalestine/comme...,44,1.702674e+09,Apparently they are reporting on CBS that the ...,1.702674e+09,IsraelPalestine
1,the difficult and ongoing work of peace at the...,16,18j82h6,https://www.reddit.com/r/IsraelPalestine/comme...,27,1.702668e+09,I just completed reading [this article from th...,1.702668e+09,IsraelPalestine
2,I think Israel will be attacked again soon,11,18j7x6d,https://www.reddit.com/r/IsraelPalestine/comme...,64,1.702668e+09,The lack of any aggressive response from the t...,1.702668e+09,IsraelPalestine
3,"As an Israeli leftist, my generation's discuss...",166,18j5trn,https://www.reddit.com/r/IsraelPalestine/comme...,245,1.702663e+09,"I'm 17, born and raised in Tel Aviv. Always be...",1.702663e+09,IsraelPalestine
4,Whats the point of issuing visa bans on violen...,18,18j51qb,https://www.reddit.com/r/IsraelPalestine/comme...,60,1.702661e+09,I mean...its not like these settlers want to g...,1.702661e+09,IsraelPalestine


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      10000 non-null  object 
 1   score      10000 non-null  int64  
 2   id         10000 non-null  object 
 3   url        10000 non-null  object 
 4   comms_num  10000 non-null  int64  
 5   created    10000 non-null  float64
 6   body       10000 non-null  object 
 7   timestamp  10000 non-null  float64
 8   subreddit  10000 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [8]:
#the csv names to be changed based on sub reddit we use, i used the following subreddit for data sourcing
#“IsraelUnderAttack”, “IsraelHamas”, “IsraelPalestineWar_23”, hamasdeadlyattack”, “IsraelPalestine” ,“IsraelWar”.
df.to_csv('IsraelUnderAttack_reddit_posts.csv', index=False)